# Notebook to add a variable to the netcdf file with integrated parameters 

## Importing

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd

import os

from tqdm import tqdm


## Datasets Preparation

In [ ]:
def datasets_preparation():

    ds_name = ('/results2/SalishSea/nowcast-green.202111/' + i[1] + '/SalishSea_1d_' + '20' + str(i[1][5:7]) + str(dict_month[i[1][2:5]])+str(i[1][0:2]) + '_' + '20' + str(i[1][5:7]) + str(dict_month[i[1][2:5]]) + str(i[1][0:2]) + '_grid_T.nc')
    ds_prod_name = ('/results2/SalishSea/nowcast-green.202111/' + i[1] + '/SalishSea_1d_'  + '20' + str(i[1][5:7]) + str(dict_month[i[1][2:5]])+str(i[1][0:2]) + '_' + '20' + str(i[1][5:7]) + str(dict_month[i[1][2:5]]) + str(i[1][0:2]) + '_prod_T.nc')
    
    ds = xr.open_dataset (ds_name)
    ds_prod = xr.open_dataset (ds_prod_name)

    flag_pr_i = (((ds_prod.PPPHY.where(mask==1)[0,0:27] * ds.e3t.where(mask==1)
        [0,0:27]).sum('deptht', skipna = True, min_count = 27) / mesh.gdepw_0[0,27])) / flag_i

    return (flag_pr_i)


## Datasets Preparation 2

In [ ]:
def datasets_preparation2(name, units):

    dataset = xr.DataArray(coords=coords, dims = ['time_counter', 'y', 'x'],
        attrs=dict(description='Integrated quantity of interest',
        long_name =name,
        units=units))
        
    return (dataset)

## File Creation

In [ ]:
def file_creation(variable, name):

    temp = variable.to_dataset(name=name)
    temp.to_netcdf(path='/data/ibougoudis/MOAD/files/integrated_model_var_old.nc', mode='a', encoding={name:{"zlib": True, "complevel": 9}})


## Data Collection

In [ ]:
dict_month = {'jan': '01',
    'feb': '02',
    'mar': '03',
    'apr': '04',
    'may': '05',
    'jun': '06',
    'jul': '07',
    'aug': '08',
    'sep': '09',
    'oct': '10',
    'nov': '11',
    'dec': '12'}

path = os.listdir('/results2/SalishSea/nowcast-green.202111/')

folders = [x for x in path if ((x[2:5]=='mar' or x[2:5]=='apr' or (x[2:5]=='feb' and x[0:2] > '14')) and (x[5:7]<'24'))]
indx_dates=(np.argsort(pd.to_datetime(folders, format="%d%b%y")))
folders = [folders[i] for i in indx_dates]

# Open the mesh mask
mesh = xr.open_dataset('/home/sallen/MEOPAR/grid/mesh_mask202108.nc')
mask = mesh.tmask.to_numpy()

file = xr.open_dataset('/data/ibougoudis/MOAD/files/integrated_model_var_old.nc')
ref = file.Flagellate

coords = dict(time_counter=file.time_counter, y=file.y, x=file.x) 

flag_pr_all = datasets_preparation2('Primary Production of Flagellates', '[mmol N m-2 s-1]')

file = xr.open_dataset('/data/ibougoudis/MOAD/files/integrated_model_var_old.nc')
ref = file.Flagellate

for i in tqdm(enumerate(folders)):

    flag_i = ref[i[0]]
    flag_pr = datasets_preparation()

    flag_pr_all[i[0]] = flag_pr

# Calling file creation

file_creation(flag_pr_all, 'Flagellate_Production_Rate')
